In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.datasets
import torchvision.transforms as transforms

import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [ ]:

transform = transforms.Compose([
    transforms.ToTensor(),                   # pixel value를 0 ~ 1 사이 값으로 변환
    transforms.Normalize((0.5, ), (0.5, )),  # 0 ~ 1 사이로 변환된 pixel value를 -1 ~ 1 사이로 변환
])

In [ ]:
trainset = torchvision.datasets.CIFAR10('./data',
                                        train=True,
                                        download=True,
                                        transform=transform)

testset = torchvision.datasets.CIFAR10('./data',
                                       train=False,
                                       download=True,
                                       transform=transform)

In [ ]:
train_loader = DataLoader(trainset,
                          batch_size=128,
                          shuffle=True,
                          num_workers=4)

test_loader = DataLoader(testset,
                          batch_size=128,
                          shuffle=True,
                          num_workers=4)

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [ ]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        
        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=96, kernel_size=3, stride=1)
        self.conv_layer2 = nn.Conv2d(in_channels=96, out_channels=256, kernel_size=3, stride=1)
        # self.conv_layer3 = nn.Conv2d(in_channels=96, out_channels=256, kernel_size=3, stride=1)
        # self.conv_layer4 = nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, stride=1)
        self.fc_layer1 = nn.Linear(in_features=5*5*256, out_features=1000) 
        self.fc_layer2 = nn.Linear(in_features=1000, out_features=10)
        
    def forward(self, x):
        x = self.conv_layer1(x)     # 32x32 -> 30x30
        x = F.relu(x)
        x = F.max_pool2d(input=x, kernel_size=3, stride=2)  # 30x30 -> 14x14
        
        x = self.conv_layer2(x)     # 14x14 -> 12x12
        x = F.relu(x)
        x = F.max_pool2d(input=x, kernel_size=3, stride=2)  # 12x12 -> 5x5
        
        # x = self.conv_layer3(x)
        # x = F.relu(x)
        # x = F.max_pool2d(input=x, kernel_size=3, stride=2)ß
        
        x = nn.Flatten(x)
        x = F.relu(self.fc_layer1(x))
        return F.relu(self.fc_layer2(x))

In [ ]:
my_model = MyModel().to(device)
my_model

my_model(
  (conv_layer1): Conv2d(3, 96, kernel_size=(3, 3), stride=(1, 1))
  (conv_layer2): Conv2d(96, 256, kernel_size=(3, 3), stride=(1, 1))
  (fc_layer1): Linear(in_features=6400, out_features=1000, bias=True)
  (fc_layer2): Linear(in_features=1000, out_features=10, bias=True)
)

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.SGD(my_model.parameters(),lr=.001)

In [ ]:
def train(model, train_loader, optimizer, losses, corrects):
    model.train()
    correct = 0
    total = len(train_loader.dataset)
    for batch, (image, label) in enumerate(train_loader):
        image, label = image.to(device), label.to(device)
        optimizer.zero_grad()
        pred_label = model(image)
        
        _, predicted = torch.max(pred_label.data, 1)
        correct += (predicted == label).sum().item()
        
        loss = criterion(pred_label, label)
        loss.backward()
        optimizer.step()
        
        if not(batch % 100):
            print(f"Train Epoch : {batch * len(image)} / {len(train_loader.dataset)} | Train Loss : {loss.item():.4f}")
    
    losses.append(loss.item())
    correct = 100 * correct / total
    corrects.append(100 * correct / total)
    print(f"Train Accuracy : {correct:.2f}")

In [ ]:
def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    total = len(test_loader.dataset)
    with torch.no_grad():
        for image, label in test_loader:
            image, label = image.to(device), label.to(device)
            pred_label = model(image)
            test_loss += criterion(pred_label, label).item()
            _, predicted = torch.max(pred_label.data, 1)
            
            correct += (predicted == label).sum().item()
            
    test_loss /= total
    correct = 100 * correct / total
            
    return test_loss, correct

In [ ]:
epochs = 100
losses = []
corrects = []
best_score = 0
for epoch in range(epochs):
    train(my_model, train_loader, optimizer, losses, corrects)
    test_loss, correct = test(my_model, test_loader)
    print(f"Epoch : {epoch + 1} | Test Loss : {test_loss:.4f} | Test Accuracy : {correct:.2f}")